This file uses cleaned poi and sod data in ft_poi_sod_clean.ipynb (second section) and for each observation in sod, finds the closest one in poi using lat and long

In [1]:
import time
from tqdm import tqdm
import requests
import pandas as pd
import os
from io import BytesIO
import gzip
from fuzzywuzzy import fuzz
from global_variables import *
from geopy.distance import geodesic
import placekey as pk

In [3]:
poi_file = "C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/poi_with_usps_address.csv.gz" 
sod_file = "C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/sod_data_branches_with_usps_address.csv.gz" 
poi = pd.read_csv(poi_file, compression='gzip')
poi = poi.drop_duplicates()
sod = pd.read_csv(sod_file, compression='gzip')
sod = sod.drop_duplicates(subset='usps_address', keep='first')

In [ ]:
poi.info()

In [ ]:
sod.info()

In [4]:
poi_sod_exact_match = poi.merge(sod[['usps_address', 'NAMEBR', 'NAMEFULL', 'SIMS_LATITUDE', 'SIMS_LONGITUDE', 'UNINUMBR', 'CERT','full_address_sod']], on='usps_address', how='inner')
poi_sod_exact_match = poi_sod_exact_match.dropna(subset=['usps_address'])
poi_sod_exact_match = poi_sod_exact_match.drop_duplicates(subset=['PLACEKEY', 'UNINUMBR'])
# poi_sod_exact_match = poi_sod_exact_match[poi_sod_exact_match.apply(lambda x: x['NAMEFULL'].lower()[0] == x['LOCATION_NAME'].lower()[0], axis=1)]

In [86]:
poi_sod_exact_match.info()

<class 'pandas.core.frame.DataFrame'>
Index: 77234 entries, 0 to 143128
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   level_0           77234 non-null  int64  
 1   index             77234 non-null  int64  
 2   CATEGORY_TAGS     63104 non-null  object 
 3   CITY              77234 non-null  object 
 4   LATITUDE          77234 non-null  float64
 5   LONGITUDE         77234 non-null  float64
 6   NAICS_CODE        77234 non-null  float64
 7   PLACEKEY          77234 non-null  object 
 8   POSTAL_CODE       77234 non-null  int64  
 9   REGION            77234 non-null  object 
 10  STREET_ADDRESS    77234 non-null  object 
 11  SUB_CATEGORY      77234 non-null  object 
 12  TOP_CATEGORY      77234 non-null  object 
 13  LOCATION_NAME     77234 non-null  object 
 14  full_address_poi  77234 non-null  object 
 15  usps_contents     77234 non-null  object 
 16  usps_address      77234 non-null  object 
 1

In [7]:
sod_unmatched = sod[~sod['UNINUMBR'].isin(poi_sod_exact_match['UNINUMBR'])]

sod_unmatched = sod_unmatched.copy()
sod_unmatched['match_string'] = sod_unmatched['ZIPBR'].astype(str)  + sod_unmatched['ADDRESBR'].str[:3] #+ sod_unmatched['NAMEFULL'].str[:2]
sod_unmatched.loc[:, 'match_string'] = sod_unmatched['match_string'].str.lower()

In [8]:
poi_sod_exact_match = poi_sod_exact_match[["CERT", "UNINUMBR", "NAMEFULL", "usps_address", "full_address_sod", "LOCATION_NAME", "PLACEKEY"]]
poi_sod_exact_match.loc[:, 'match_type'] = 'exact'

In [9]:
sod_unmatched.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17642 entries, 0 to 90923
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   level_0           17642 non-null  int64  
 1   index             17642 non-null  int64  
 2   YEAR              17642 non-null  int64  
 3   CERT              17642 non-null  int64  
 4   BRNUM             17642 non-null  int64  
 5   UNINUMBR          17642 non-null  int64  
 6   NAMEFULL          17642 non-null  object 
 7   ADDRESBR          17642 non-null  object 
 8   CITYBR            17642 non-null  object 
 9   CNTYNAMB          17642 non-null  object 
 10  STALPBR           17642 non-null  object 
 11  ZIPBR             17642 non-null  int64  
 12  NAMEBR            17642 non-null  object 
 13  SIMS_LATITUDE     17642 non-null  float64
 14  SIMS_LONGITUDE    17642 non-null  float64
 15  full_address_sod  17642 non-null  object 
 16  usps_contents     17642 non-null  object 
 17

In [10]:
def geo_to_placekey(row):
    return pk.geo_to_placekey(row['SIMS_LATITUDE'], row['SIMS_LONGITUDE'])

In [11]:
sod_unmatched['DERIVED_PLACEKEY'] = sod_unmatched.apply(geo_to_placekey, axis=1)

In [12]:
all_us_poi_csv_path = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/all_us_poi.csv'
poi_all_us = pd.read_csv(all_us_poi_csv_path+'.gz', compression='gzip')

poi_all_us = poi_all_us.dropna(subset=['POSTAL_CODE'])

poi_all_us['POSTAL_CODE'] = poi_all_us['POSTAL_CODE'].astype(int)

poi_all_us['match_string'] = poi_all_us['POSTAL_CODE'].astype(str)  + poi_all_us['STREET_ADDRESS'].str[:3] #+ poi_all_us['LOCATION_NAME'].str[:2]
poi_all_us.loc[:, 'match_string'] = poi_all_us['match_string'].str.lower()

In [18]:
poi_all_us['DERIVED_PLACEKEY'] = '@'+poi_all_us['PLACEKEY'].str.split('@').str[1]

In [14]:
poi_all_us.set_index('match_string', inplace=True)

In [24]:
# Assuming you have a DataFrame named 'derived_placekey_match'

# Convert the first character of 'STREET_ADDRESS' and 'NAMEBR' to lowercase



In [43]:
derived_placekey_match = sod_unmatched.merge(poi_all_us, on='DERIVED_PLACEKEY', how='inner')

derived_placekey_match['STREET_ADDRESS'] = derived_placekey_match['STREET_ADDRESS'].str[0].str.lower()
derived_placekey_match['NAMEBR'] = derived_placekey_match['NAMEBR'].str[0].str.lower()

# Filter observations based on the conditions
derived_placekey_match = derived_placekey_match[
    (derived_placekey_match['POSTAL_CODE'] == derived_placekey_match['ZIPBR']) &
    (derived_placekey_match['STREET_ADDRESS'] == derived_placekey_match['ADDRESBR'].str[0].str.lower()) &
    (derived_placekey_match['NAMEBR'] == derived_placekey_match['LOCATION_NAME'].str[0].str.lower())
]

columns_to_keep = ["CERT", "UNINUMBR", "NAMEFULL", "usps_address", "full_address_sod", "LOCATION_NAME", "PLACEKEY"]

# Create a new DataFrame with only the specified columns
derived_placekey_match = derived_placekey_match[columns_to_keep]

derived_placekey_match.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10665 entries, 18 to 238158
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   CERT              10665 non-null  int64 
 1   UNINUMBR          10665 non-null  int64 
 2   NAMEFULL          10665 non-null  object
 3   usps_address      10665 non-null  object
 4   full_address_sod  10665 non-null  object
 5   LOCATION_NAME     10665 non-null  object
 6   PLACEKEY          10665 non-null  object
dtypes: int64(2), object(5)
memory usage: 666.6+ KB


In [44]:
with gzip.open('C:/Users/dratnadiwakara2/Downloads/temp_dewey/sod_closest_match_derived_placekey_match.csv.gz', 'wt', encoding='utf-8') as gzipped_file:
    derived_placekey_match.to_csv(gzipped_file, index=False)

In [27]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """Calculate the great-circle distance between two points on Earth (in km)."""
    coord1 = (lat1, lon1)
    coord2 = (lat2, lon2)
    return geodesic(coord1, coord2).kilometers


In [28]:
output_csv_file = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/sod_closest_match_no_name.csv'

In [29]:
i=1


with tqdm(total=len(sod_unmatched)) as pbar:
    with open(output_csv_file, 'a') as csv_file:
        for index_a, row_a in sod_unmatched.iterrows():
    
            pbar.update(1)
            try:
                min_distance = float('inf')
                closest_point = None
        
                var = row_a['match_string']
                df_b = poi_all_us.query(f"match_string == '{var}'")
        
                if len(df_b) == 0:
                   csv_file.write(f"{row_a['UNINUMBR']},,,{var}\n")
                   continue
        
                for index_b, row_b in df_b.iterrows():
                    distance = haversine_distance(row_a['SIMS_LATITUDE'], row_a['SIMS_LONGITUDE'], row_b['LATITUDE'], row_b['LONGITUDE'])
                    if distance < min_distance:
                        min_distance = distance
                        closest_point = (row_b['PLACEKEY'], distance)
        
                csv_file.write(f"{row_a['UNINUMBR']},{closest_point[0]},{closest_point[1]},{var}\n")
            except:
                pass


100%|██████████| 17642/17642 [5:59:18<00:00,  1.22s/it]  


In [29]:
# sod_closest_match.csv was constructed by matching first two characters of address, first two characters of name, and the zip code
temp1 = pd.read_csv('C:/Users/dratnadiwakara2/Downloads/temp_dewey/sod_closest_match.csv',header=None)
# sod_closest_match.csv was constructed by matching first three characters of address and the zip code
temp2 = pd.read_csv('C:/Users/dratnadiwakara2/Downloads/temp_dewey/sod_closest_match_no_name.csv',header=None)

In [45]:


partial_match = pd.concat([temp1, temp2], ignore_index=True)

partial_match.columns = ['UNINUMBR','PLACEKEY','distance','match_string']
partial_match.drop_duplicates(subset=['UNINUMBR'], inplace=True)
partial_match = partial_match[~(partial_match['distance'].isna() | (partial_match['distance'] > 0.1))]


In [46]:
partial_match = partial_match.merge(sod, on='UNINUMBR', how='inner')

In [47]:
partial_match = partial_match.merge(poi_all_us, on='PLACEKEY', how='inner')


In [48]:
partial_match = partial_match[["CERT", "UNINUMBR", "NAMEFULL", "usps_address", "full_address_sod", "LOCATION_NAME", "PLACEKEY"]].copy()
partial_match.loc[:, 'match_type'] = 'fuzzy'


In [49]:
# sod_closest_match_derived_placekey_match.csv was constructed by matching the derived placekey in sod data
temp3 = pd.read_csv('C:/Users/dratnadiwakara2/Downloads/temp_dewey/sod_closest_match_derived_placekey_match.csv.gz')
temp3.loc[:, 'match_type'] = 'derived_placekey'

In [50]:
partial_match = pd.concat([partial_match, temp3], ignore_index=True)

In [51]:
partial_match = partial_match.drop_duplicates()

In [52]:
partial_match.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14991 entries, 0 to 14990
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   CERT              14991 non-null  int64 
 1   UNINUMBR          14991 non-null  int64 
 2   NAMEFULL          14991 non-null  object
 3   usps_address      14990 non-null  object
 4   full_address_sod  14991 non-null  object
 5   LOCATION_NAME     14991 non-null  object
 6   PLACEKEY          14991 non-null  object
 7   match_type        14991 non-null  object
dtypes: int64(2), object(6)
memory usage: 937.1+ KB


In [53]:
# Concatenate partial_match and poi_sod_exact_match vertically
concatenated_df = pd.concat([partial_match, poi_sod_exact_match], ignore_index=True)


In [54]:
output_file_path = "C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/poi_sod_4.csv.gz"
with gzip.open(output_file_path, 'wt', encoding='utf-8') as gzipped_file:
    concatenated_df.to_csv(gzipped_file, index=False)